# Carga del dataset de tweets con sentimiento

In [126]:
import kagglehub
from kagglehub import KaggleDatasetAdapter

file_path = "mbsa.csv"  

# Load the latest version
df_tweets = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "gauravduttakiit/bitcoin-tweets-16m-tweets-with-sentiment-tagged",
  file_path,
)

# Eliminar columnas con Date antes de 2018
df_tweets = df_tweets[df_tweets['Date'] >= '2018-01-01']
# Eliminar columnas sin Date
df_tweets = df_tweets.dropna(subset=['Date'])

# Usar muestra de datos aleatoria de 1000 filas

df_tweets = df_tweets.sample(n=1000, random_state=1)


print(df_tweets.head())


/var/folders/x7/yqjxlcpn6896ntpm60n8wg4r0000gn/T/ipykernel_52846/1207638233.py:7: DeprecationWarning: load_dataset is deprecated and will be removed in a future version.
  df_tweets = kagglehub.load_dataset(


                Date                                               text  \
12750401  2019-08-10  ⚠️In case you were wondering if Bitcoin would ...   
6500576   2019-06-08  Imagine someone predicting back in March that ...   
1532247   2019-03-22  #Doviz\n-------------------\n#USD : 5.4740\n#E...   
17978827  2019-10-31  7 Keys to Overcoming Risk on the Path to Succe...   
6929710   2019-06-17  @membrana_io @ABCC_Exchange @ProBit_Exchange h...   

         Sentiment  
12750401  Negative  
6500576   Negative  
1532247   Negative  
17978827  Negative  
6929710   Positive  


# Cargar de dataset de precios de bitcoin

In [127]:
file_path = "btc_15m_data_2018_to_2025.csv"  

df_bitcoin = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "novandraanugrah/bitcoin-historical-datasets-2018-2024",
  file_path,
)

print(df_bitcoin.head())

/var/folders/x7/yqjxlcpn6896ntpm60n8wg4r0000gn/T/ipykernel_52846/3152041267.py:3: DeprecationWarning: load_dataset is deprecated and will be removed in a future version.
  df_bitcoin = kagglehub.load_dataset(


             Open time      Open      High       Low     Close      Volume  \
0  2018-01-01 00:00:00  13715.65  13715.65  13400.01  13556.15  123.616013   
1  2018-01-01 00:15:00  13533.75  13550.87  13402.00  13521.12   98.136430   
2  2018-01-01 00:30:00  13500.00  13545.37  13450.00  13470.41   79.904037   
3  2018-01-01 00:45:00  13494.65  13690.87  13450.00  13529.01  141.699719   
4  2018-01-01 01:00:00  13528.99  13571.74  13402.28  13445.63   72.537533   

                Close time  Quote asset volume  Number of trades  \
0  2018-01-01 00:14:59.999        1.675545e+06              1572   
1  2018-01-01 00:29:59.999        1.321757e+06              1461   
2  2018-01-01 00:44:59.999        1.078825e+06              1000   
3  2018-01-01 00:59:59.999        1.917783e+06              1195   
4  2018-01-01 01:14:59.999        9.778198e+05               898   

   Taker buy base asset volume  Taker buy quote asset volume  Ignore  
0                    63.227133                  8.5

# Asignacion de precio segun el dataset de bitcoin

In [128]:
# Por cada fila en el dataset de bitcoin, se busca la fecha en el dataset de tweets
# y se añade la columna de sentimiento al dataset de bitcoin
import datetime as DateTime
import pandas as pd



# -- Cambio en el dataset de bitcoin ----

# Convertir Open time a formato date.split(" ")[0]
df_bitcoin["Open time"] = df_bitcoin["Open time"].apply(lambda x: x.split(" ")[0])

# Eliminar fechas duplicadas en el dataset de bitcoin
df_bitcoin = df_bitcoin.drop_duplicates(subset=["Open time"])

# Ordenar fecha de menor a mayor
df_bitcoin = df_bitcoin.sort_values(by=["Open time"])

# -- Buscar precio para cada tweet ----

bitcoin_prices = {}
for index, row in df_bitcoin.iterrows():
    bitcoin_prices[row['Open time']] = row['Close']

df_tweets['Bitcoin'] = df_tweets['Date'].apply(lambda date: bitcoin_prices.get(date))

print(df_tweets.head())

                Date                                               text  \
12750401  2019-08-10  ⚠️In case you were wondering if Bitcoin would ...   
6500576   2019-06-08  Imagine someone predicting back in March that ...   
1532247   2019-03-22  #Doviz\n-------------------\n#USD : 5.4740\n#E...   
17978827  2019-10-31  7 Keys to Overcoming Risk on the Path to Succe...   
6929710   2019-06-17  @membrana_io @ABCC_Exchange @ProBit_Exchange h...   

         Sentiment   Bitcoin  
12750401  Negative  11917.06  
6500576   Negative   8004.95  
1532247   Negative   3975.49  
17978827  Negative   9182.25  
6929710   Positive   9015.74  


In [130]:

# Información general
print("Primeras 5 filas:\n", df_tweets.head())
print("\nInformación del dataset:\n", df_tweets.info())
print("\nEstadísticas descriptivas:\n", df_tweets.describe(include='all'))
print("\nConteo de valores únicos en 'Sentiment':\n", df_tweets['Sentiment'].value_counts())

Primeras 5 filas:
                 Date                                               text  \
12750401  2019-08-10  ⚠️In case you were wondering if Bitcoin would ...   
6500576   2019-06-08  Imagine someone predicting back in March that ...   
1532247   2019-03-22  #Doviz\n-------------------\n#USD : 5.4740\n#E...   
17978827  2019-10-31  7 Keys to Overcoming Risk on the Path to Succe...   
6929710   2019-06-17  @membrana_io @ABCC_Exchange @ProBit_Exchange h...   

         Sentiment   Bitcoin  
12750401  Negative  11917.06  
6500576   Negative   8004.95  
1532247   Negative   3975.49  
17978827  Negative   9182.25  
6929710   Positive   9015.74  
<class 'pandas.core.frame.DataFrame'>
Index: 1000 entries, 12750401 to 7980243
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       1000 non-null   object 
 1   text       1000 non-null   object 
 2   Sentiment  961 non-null    object 
 3   Bitcoin    1000 non-null   f